# Setup

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
import yaml
import os

In [8]:
with open('config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

In [9]:
openai = ChatOpenAI(model_name='gpt-4o', temperature=0)

# SequentialChain

In [10]:
# Carregar os documentos
loader = TextLoader('base_conhecimento_britadeira.txt')
documents = loader.load()
# Carregar histórico de conversas 
historico_conversas = """Cliente: Minha britadeira não liga. Chatbot: Você já verificou 
                         se a bateria está carregada e conectada corretamente?"""
# Pergunta do cliente
pergunta = "Minha britadeira não liga. Eu já veriquei e a bateria está carregada e conectada corretamente"

In [11]:
inputs = {
    "context": "\n".join(doc.page_content for doc in documents),
    "question": pergunta,
    "historico": historico_conversas
}

In [12]:
prompt_base_conhecimento = PromptTemplate(
    input_variables=["context", "question"],
    template="""Use o seguinte contexto para responder à pergunta. 
    Responda apenas com base nas informações fornecidas.
    Não forneceça instruções de procedimento já realizados.
    Não utilize informações externas ao contexto:
    Contexto: {context}
    Pergunta: {question}"""
)
prompt_historico_conversas = PromptTemplate(
    input_variables=["historico", "question"],
    template="""Use o histórico de conversas para responder à pergunta. 
    Responda apenas com base nas informações fornecidas. 
    Não forneceça instruções de procedimento já realizados.
    Não utilize informações externas ao contexto:
    Histórico: {historico}
    Pergunta: {question}"""
)
prompt_final = PromptTemplate(
    input_variables=["resposta_base_conhecimento", "resposta_historico_conversas"],
    template="""Combine as seguintes respostas para gerar uma resposta final,
    mas não forneça instruções de procedimentos já realizados:
    Resposta da base de conhecimento: {resposta_base_conhecimento}
    Resposta do histórico de conversas: {resposta_historico_conversas}"""
)

In [13]:
print(prompt_base_conhecimento)

input_variables=['context', 'question'] template='Use o seguinte contexto para responder à pergunta. \n    Responda apenas com base nas informações fornecidas.\n    Não forneceça instruções de procedimento já realizados.\n    Não utilize informações externas ao contexto:\n    Contexto: {context}\n    Pergunta: {question}'


In [14]:
# Definir as cadeias  
chain_base_conhecimento = prompt_base_conhecimento | openai
chain_historico_conversas = prompt_historico_conversas | openai
chain_final = prompt_final | openai

In [15]:
print(chain_base_conhecimento)

first=PromptTemplate(input_variables=['context', 'question'], template='Use o seguinte contexto para responder à pergunta. \n    Responda apenas com base nas informações fornecidas.\n    Não forneceça instruções de procedimento já realizados.\n    Não utilize informações externas ao contexto:\n    Contexto: {context}\n    Pergunta: {question}') last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7f1a30f54350>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f1a30f56650>, model_name='gpt-4o', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')


In [16]:
print(chain_historico_conversas)

first=PromptTemplate(input_variables=['historico', 'question'], template='Use o histórico de conversas para responder à pergunta. \n    Responda apenas com base nas informações fornecidas. \n    Não forneceça instruções de procedimento já realizados.\n    Não utilize informações externas ao contexto:\n    Histórico: {historico}\n    Pergunta: {question}') last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7f1a30f54350>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f1a30f56650>, model_name='gpt-4o', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')


In [17]:
# Passando dados e executando
resultado_base_conhecimento = chain_base_conhecimento.invoke({"context": inputs["context"], "question": inputs["question"]})
resultado_historico_conversas = chain_historico_conversas.invoke({"historico": inputs["historico"], "question": inputs["question"]})
resultado_final = chain_final.invoke({"resposta_base_conhecimento": resultado_base_conhecimento, 
                                      "resposta_historico_conversas": resultado_historico_conversas})

In [18]:
print("Resultado Base de Conhecimento:\n", resultado_base_conhecimento)
print("----")
print("Resultado Histórico de Conversas:\n", resultado_historico_conversas)

Resultado Base de Conhecimento:
 content='Aguarde 30 segundos após conectar a bateria para que o sistema inicialize. Verifique se o interruptor liga/desliga está na posição correta. Se o problema persistir, contate o suporte técnico pelo 0800 555 5555.' response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 681, 'total_tokens': 734, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'finish_reason': 'stop', 'logprobs': None} id='run-d0203704-dac9-4154-bc6d-2b3e9028e59d-0' usage_metadata={'input_tokens': 681, 'output_tokens': 53, 'total_tokens': 734}
----
Resultado Histórico de Conversas:
 content='Pode haver outros problemas, como um fusível queimado ou um problema no interruptor.' response_metadata={'token_usage': {'completion_tokens': 19

In [19]:
print(resultado_final.content)

Aguarde 30 segundos após conectar a bateria para que o sistema inicialize. Verifique se o interruptor liga/desliga está na posição correta. Pode haver outros problemas, como um fusível queimado ou um problema no interruptor. Se o problema persistir, contate o suporte técnico pelo 0800 555 5555.


In [20]:
print(resultado_final)

content='Aguarde 30 segundos após conectar a bateria para que o sistema inicialize. Verifique se o interruptor liga/desliga está na posição correta. Pode haver outros problemas, como um fusível queimado ou um problema no interruptor. Se o problema persistir, contate o suporte técnico pelo 0800 555 5555.' response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 488, 'total_tokens': 559, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90122d973c', 'finish_reason': 'stop', 'logprobs': None} id='run-a1ac73e5-4705-47e6-8b50-695af0452eef-0' usage_metadata={'input_tokens': 488, 'output_tokens': 71, 'total_tokens': 559}
